<a href="https://colab.research.google.com/github/cangokalp/char_rnn_rap_generator/blob/master/char_rnn_rap_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/cangokalp/char_rnn_rap_generator/blob/master/char_rnn_rap_generator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [1]:
import pdb
import pickle
import re

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
import os
import sys

     |████████████████████████████████| 1.0MB 2.8MB/s 


In [0]:
#NOTE TO WILL: 
#Go to my github https://github.com/cangokalp/char_rnn_rap_generator
#and download the two files; lyrics_array_mix.pickle and rap_bot_v2_model.h5, they are under data and saved model folders, respectively.
#After that run the below cell. 
#Then on the tab to the left of the screen there is a folder icon, 
#click that and youll see data and saved_models folders. 
#click upload and upload the files you downloaded. 
#Then drag them in their corresponding folders
#then you can continue

In [108]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# DRIVE_PATH = '/content/gdrive/My Drive'

PROJECT_ROOT_DIR = "."
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data")
os.makedirs(DATA_PATH, exist_ok=True)

MODEL_PATH = os.path.join(PROJECT_ROOT_DIR, "saved_models")
os.makedirs(MODEL_PATH, exist_ok=True)

def save(fname, data extension="pickle"):
    path = os.path.join(DATA_PATH, fname + "." + extension)
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load(fname, extension='pickle'):
    path = os.path.join(DATA_PATH, fname + "." + extension)
    
    with open(path, 'rb') as f:
        item = pickle.load(f)

    return item

SyntaxError: ignored

In [0]:
#GRAB TRAINING DATA:
documents = load('lyrics_array_mix')

In [0]:
#CREATE NEW TRAINING DATA - IF YOU WANT

import time
!pip install lyricsgenius
import lyricsgenius

API_TOKEN = '1OWmIr37ZuUI5hDXjN_FIXJlbZadIqH6JBDObBCiF_MNySn4BQqh8aQ8OUYF5muk'
fname = 'mix'

#### grab the lyrics #####
artists = []
genius = lyricsgenius.Genius(API_TOKEN, skip_non_songs=True, remove_section_headers=True)

lonely = genius.search_artist("The Lonely Island", max_songs=80, sort="popularity")
artists.append(lonely)
time.sleep(30)

fetty = genius.search_artist("Fetty Wap", max_songs=15, sort="popularity")
artists.append(fetty)
childish = genius.search_artist("Childish Gambino", max_songs=20, sort="popularity")
artists.append(childish)
time.sleep(10)

drake = genius.search_artist("Drake", max_songs=20, sort="popularity")
artists.append(drake)
kendrick = genius.search_artist("Kendrick Lamar", max_songs=20, sort="popularity")
artists.append(kendrick)
time.sleep(20)

drake = genius.search_artist("Lil Dicky", max_songs=20, sort="popularity")
artists.append(drake)
chet = genius.search_artist("Chet Faker", max_songs=10, sort="popularity")
artists.append(chet)
time.sleep(30)

snoop = genius.search_artist("Snoop Dogg", max_songs=15, sort="popularity")
artists.append(snoop)
travis = genius.search_artist("Travis Scott", max_songs=10, sort="popularity")
artists.append(travis)
q = genius.search_artist("ScHoolboy Q", max_songs=10, sort="popularity")
artists.append(q)
wayne = genius.search_artist("Lil Wayne", max_songs=15, sort="popularity")
artists.append(wayne)
time.sleep(30)

eminem = genius.search_artist("Eminem", max_songs=10, sort="popularity")
artists.append(eminem)
ariana = genius.search_artist("Ariana Grande", max_songs=5, sort="popularity")
artists.append(ariana)
wiz = genius.search_artist("Wiz Khalifa", max_songs=10, sort="popularity")
artists.append(wiz)
beatles = genius.search_artist("The Beatles", max_songs=15, sort="popularity")
artists.append(beatles)
billie = genius.search_artist("Billie Eilish", max_songs=15, sort="popularity")
artists.append(billie)
peas = genius.search_artist("Black Eyed Peas", max_songs=10, sort="popularity")
artists.append(peas)
time.sleep(40)

chance = genius.search_artist("Chance The Rapper", max_songs=10, sort="popularity")
artists.append(chance)
kanye = genius.search_artist("Kanye West", max_songs=15, sort="popularity")
artists.append(kanye)
outkast = genius.search_artist("OutKast", max_songs=10, sort="popularity")
artists.append(outkast)
rihanna = genius.search_artist("Rihanna", max_songs=10, sort="popularity")
artists.append(rihanna)


documents = []
for artist in artists:
  for song in artist.songs: 
    documents.append(song.lyrics)

save('lyrics_array_' + fname, documents)

In [0]:
# COMBINE ALL SONGS INTO ONE TEXT FILE

comedy_rap_text = ''
documents = documents[81:]
documents = documents[75:96]
for document in documents:
  if document is not None:
    comedy_rap_text += document + '\n'


In [0]:
# CLEAN THE TEXT, GET RID OF SOME PATTERNS

comedy_rap_prepped = comedy_rap_text
import re
pattern = re.compile(r'[(.?)]')
comedy_rap_prepped = re.sub(pattern, '', comedy_rap_prepped)

In [0]:
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
#   print('and then re-execute this cell.')
# else:
#   print(gpu_info)

In [0]:
# PREPROCESSING THE TRAINING DATA SO THAT CHAR MODEL CAN USE

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(comedy_rap_prepped)
max_id = len(tokenizer.word_index) 
dataset_size = tokenizer.document_count

[encoded] = np.array(tokenizer.texts_to_sequences([comedy_rap_prepped])) - 1

train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 60) (32, 100)


In [0]:
# IF YOU WANT TO USE ALREADY TRAIN MODEL - rap_bot_v2
model = keras.models.load_model('./saved_models/rap_bot_v2_model.h5')

In [0]:
# IF YOU WANT TO TRAIN FROM SCRATCH - RNN NETWORK WITH DROPOUT
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")

In [0]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)

In [0]:
# TO SAVE THE TRAINED NEW MODEL
model.save('./saved_models/rap_bot_v3_model.h5')

In [0]:
# HELPER FUNCTIONS FOR NEW LYRICS GENERATION

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]


def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text


In [0]:
# THIS WILL GENERATE WARNINGS - JUST IGNORE
# temperature param represents how the next word is sampled, close to 0 just does argmax, 
# around 1 worked good for me you can just try different and look at the results
# to not to keep generating same things usually they dont do argmax over the probabilities instead you sample from them with the prob of the probabilities
text = complete_text("c", temperature=1, n_chars=400)

In [0]:
# !!!!! LETS SEEE :D
print(text)

gets fucked a mouth now
and i'm a boat rorl like beaciena back
this one with the sun more nice
401k with a boat, i'm on a boat

you couldn't even even  fucked out of the ass seven shawty

old my bluesanimair, hey she say, i'm around me on a boat
wear this i love the lad-elfe with the girl
she can come to look, i'm so daddy
i'm on that 9, let's get s
